# Build Treasury Quotes CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

### Note on the raw data

The `ITYPE` column identifies
* 1: bonds
* 2: notes
* 4: bills
* 11: TIPS notes
* 12: TIPS bonds

Also
* The column `TDPUBOUT` lists the dollar size of the issue (public debt outstanding.)
* `TMATDT` column lists the maturity date.

#### Data Guide

For more on the original data source, see the CRSP documentation

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')

In [4]:
DATE = '2024-07-15'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204086.0,20260215.106000,2024-07-15,102.390625,102.421875,102.406250,M,I,2.489011,0.000174,0.000119,549.161842,6194.0,12838.0,0.0,NaN,None
1,204090.0,20270815.106370,2024-07-15,106.335938,106.382812,106.359375,M,I,2.644574,0.000267,0.000113,1018.164310,3633.0,9197.0,0.0,NaN,None
2,204097.0,20310215.105370,2024-07-15,107.539062,107.585938,107.562500,M,I,2.229739,-0.001586,0.000110,2030.965676,8110.0,16428.0,0.0,NaN,None
3,204099.0,20370215.104750,2024-07-15,105.101562,105.164062,105.132812,M,I,1.970467,-0.004140,0.000114,3479.266031,4976.0,16589.0,0.0,NaN,None
4,204101.0,20380215.104370,2024-07-15,101.007812,101.070312,101.039062,M,I,1.814904,-0.004566,0.000116,3733.499899,6758.0,22525.0,0.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,208239.0,20241219.400000,2024-07-15,97.806361,97.815083,97.810722,M,I,0.000000,0.000542,0.000141,157.000000,NaN,70333.0,0.0,NaN,None
443,208240.0,20250102.400000,2024-07-15,97.627375,97.629750,97.628563,M,I,0.000000,0.000538,0.000140,171.000000,NaN,NaN,0.0,NaN,None
444,208254.0,20250710.400000,2024-07-15,95.385000,95.395000,95.390000,M,I,0.000000,0.000615,0.000131,360.000000,NaN,NaN,0.0,NaN,None
445,208256.0,20270715.204370,2024-07-15,100.371094,100.402344,100.386719,M,I,0.000000,-0.000233,0.000115,1038.273617,NaN,NaN,0.0,NaN,None


In [5]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,20241115.107500,912810ES,1994-05-15,2024-11-15,0.0,7.500,2.0,3.7500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204084.0,20250215.107620,20250215.107620,912810ET,1995-02-15,2025-02-15,0.0,7.625,2.0,3.8125,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204085.0,20250815.106870,20250815.106870,912810EV,1995-08-15,2025-08-15,0.0,6.875,2.0,3.4375,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204086.0,20260215.106000,20260215.106000,912810EW,1996-02-15,2026-02-15,0.0,6.000,2.0,3.0000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204087.0,20260815.106750,20260815.106750,912810EX,1996-08-15,2026-08-15,0.0,6.750,2.0,3.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,208251.0,20250109.400000,20250109.400000,912797LX,2024-07-11,2025-01-09,0.0,0.000,0.0,0.0000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
443,208252.0,20250116.400000,20250116.400000,912797LY,2024-07-18,2025-01-16,0.0,0.000,0.0,0.0000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
444,208254.0,20250710.400000,20250710.400000,912797LW,2024-07-11,2025-07-10,0.0,0.000,0.0,0.0000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
445,208256.0,20270715.204370,20270715.204370,91282CKZ,2024-07-15,2027-07-15,0.0,4.375,2.0,2.1875,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


In [6]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')
rawdata

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204086.0,20260215.106000,2024-07-15,102.390625,102.421875,102.406250,M,I,2.489011,0.000174,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204090.0,20270815.106370,2024-07-15,106.335938,106.382812,106.359375,M,I,2.644574,0.000267,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204097.0,20310215.105370,2024-07-15,107.539062,107.585938,107.562500,M,I,2.229739,-0.001586,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204099.0,20370215.104750,2024-07-15,105.101562,105.164062,105.132812,M,I,1.970467,-0.004140,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204101.0,20380215.104370,2024-07-15,101.007812,101.070312,101.039062,M,I,1.814904,-0.004566,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,208239.0,20241219.400000,2024-07-15,97.806361,97.815083,97.810722,M,I,0.000000,0.000542,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
443,208240.0,20250102.400000,2024-07-15,97.627375,97.629750,97.628563,M,I,0.000000,0.000538,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
444,208254.0,20250710.400000,2024-07-15,95.385000,95.395000,95.390000,M,I,0.000000,0.000615,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
445,208256.0,20270715.204370,2024-07-15,100.371094,100.402344,100.386719,M,I,0.000000,-0.000233,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


***

# Process Data

In [7]:
import sys
sys.path.insert(0, '../cmds')
from wrds_cmds import process_wrds_treasury_data, select_maturities

data = process_wrds_treasury_data(rawdata)

In [8]:
idx = select_maturities(data)
data_select = data.loc[idx]

***

# Save to Excel

In [9]:
### raw data
outfile = f'../data/treasury_quotes_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    rawdata.to_excel(writer, sheet_name= 'quotes', index=False)


### processed data
outfile = f'../data/treasury_quotes_clean_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=True)
    data_select.to_excel(writer, sheet_name = 'selected quotes', index=True)